In [1]:
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import pdftotext
from collections import Counter
import numpy as np
from nltk.util import ngrams

## PLAN DZIAŁANIA <br>
1. Tworzę Counter ze wszystkimi n-gramami (o danej długości i o danych licznościach) <br>
2. Tworzę listę ze wszystkimi wyrazami które są w Counterze <br>
3. Dla każdego wyrazu wyliczam o ile za dużo jest go w Counterze i tą liczbę potem będę odejmować w macierzy

In [2]:
def n_grams(text, n=2):
    '''
    wymagania: modul re
    text - artykul, na podstawie ktorego chcemy uzyskac liste n gramow
    zalozenie - 1.text zostal wczesniej poddany zabiegom lematyzacji i czyszczenia
                2.type(text)==str
    n>2, type(n)==int
    '''
    if type(text) != str:
        raise NameError("Zly typ argumentu text, text musi byc typu str")
    if type(n) != int or n<2:
        raise NameError("n musi byc liczba calkowita, n>2")
    
    
    text = text.lower()
    tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', text).split() #slowa z text zapisane do listy
    
    n_grams=list() #miejsce na n gramy
    
    for i in range(len(tokens)-n+1):
        s = tokens[i] 
        for j in range(1,n):
            s = s + "_" + tokens[i+j]
        n_grams.append(s)
    
    return n_grams
    

In [3]:
file_paths = ['/home/kasia/Pulpit/LD_data/HoangLongThuy.pdf'] #Tu będę miała pobraną 

In [4]:
#Element listy zawiera pełny tekst jednego z artykułów
text_list = []
for text in file_paths:
    with open(text, "rb") as f:
        pdf = pdftotext.PDF(f)
        el_of_list = ''
        #Ponieważ page in pdf  - to jest strona z artykułu to łącze stringi, pewnie to można lepiej
        for page in pdf:
            el_of_list = el_of_list+page
        text_list.append(el_of_list)

In [5]:
#Zakładam długość ngramu=7 
n_gram = 7
#Zakładam, że liczba powtórzeń tych n-gramów to co najmniej 4
count_n_gram=4

In [6]:
#Główny słownik
main_dict = Counter(dict(filter(lambda x: x[1] >=count_n_gram, Counter(n_grams(text_list[0],n_gram)).items())))
main_dict

Counter({'eai_endorsed_transactions_on_scalable_information_systems': 6,
         'research_on_innovating_applying_multiple_paths_routing': 4,
         'on_innovating_applying_multiple_paths_routing_technique': 4,
         'innovating_applying_multiple_paths_routing_technique_based': 4,
         'applying_multiple_paths_routing_technique_based_on': 4,
         'multiple_paths_routing_technique_based_on_fuzzy': 4,
         'paths_routing_technique_based_on_fuzzy_logic': 4,
         'routing_technique_based_on_fuzzy_logic_and': 4,
         'technique_based_on_fuzzy_logic_and_genetic': 4,
         'based_on_fuzzy_logic_and_genetic_algorithm': 4,
         'on_fuzzy_logic_and_genetic_algorithm_for': 4,
         'fuzzy_logic_and_genetic_algorithm_for_routing': 4,
         'logic_and_genetic_algorithm_for_routing_messages': 4,
         'and_genetic_algorithm_for_routing_messages_in': 4,
         'genetic_algorithm_for_routing_messages_in_service': 4,
         'algorithm_for_routing_messages_i

In [7]:
#Zbieram teraz wszystkie wyrazy z countera
stri =""
words_list =[]
for i in np.unique(list(main_dict.elements())):
    #Ta podłoga bo tak są połączone wyrazy
    stri = stri + i + "_"
words_list = np.unique(stri.split("_"))

In [8]:
#Teraz dla każdego wyrazu z words_list policzę o ile za dużo występuje w Counterze
words_list

array(['', '01', '02', '2', '2015', '4', 'al', 'algorithm', 'and',
       'applying', 'based', 'e2', 'eai', 'endorsed', 'et', 'for', 'fuzzy',
       'genetic', 'in', 'information', 'innovating', 'issue', 'logic',
       'long', 'messages', 'multiple', 'n', 'on', 'oriented', 'paths',
       'research', 'routing', 'scalable', 'service', 'systems', 't',
       'technique', 'transactions', 'volume'], dtype='<U12')

In [9]:
num_to_sub = {}

In [10]:
#Stworzę nowy counter z elementami, które zawierają i:
for i in words_list[1:]:

    count_i = Counter()
    for j in main_dict:
            #Żeby nie policzyło np. dla in: routing
        if ("_"+i+"_" in j or (i[-len(i):-1]==j[-len(i):-1] and i[-1]==j[-1])or i[0:len(i)]+"_"==j[0:len(i)+1] ):
            count_i[j] = main_dict[j]
        #count_i - counter na którym pracuję
    counter_values= list(range(count_n_gram,max(count_i.items(), key=lambda x: x[1])[1]+1))

    lista_counterow = []
        #Długości powyższych counterów
    len_counter = []
    count_val = []

    iterator = 0
    for k in counter_values:
        if  Counter(dict(filter(lambda x: x[1] ==k, count_i.items())))!=Counter():
            lista_counterow.append(Counter(dict(filter(lambda x: x[1] == k, count_i.items()))))

            len_counter.append(len(lista_counterow[iterator]))
            count_val.append(k)
            iterator += 1


    list_to_mult = [x - 1 for x in count_val]
    new = np.array(list_to_mult)*np.array(len_counter)
    N = sum(np.array(count_val)*np.array(len_counter))
    
    num_to_sub[i]=N
   

In [11]:
num_to_sub

{'01': 26,
 '02': 34,
 '2': 40,
 '2015': 38,
 '4': 36,
 'al': 4,
 'algorithm': 28,
 'and': 28,
 'applying': 16,
 'based': 28,
 'e2': 28,
 'eai': 6,
 'endorsed': 6,
 'et': 8,
 'for': 28,
 'fuzzy': 28,
 'genetic': 28,
 'in': 16,
 'information': 18,
 'innovating': 12,
 'issue': 38,
 'logic': 28,
 'long': 12,
 'messages': 20,
 'multiple': 20,
 'n': 28,
 'on': 42,
 'oriented': 8,
 'paths': 24,
 'research': 4,
 'routing': 52,
 'scalable': 12,
 'service': 12,
 'systems': 24,
 't': 16,
 'technique': 28,
 'transactions': 6,
 'volume': 42}